In [36]:
import pandas as pd

In [37]:
submission = {
    'PassengerId': pd.read_csv('data/test.csv').PassengerId.values,
    'Survived': 1  # Baseline. Let it be that everyone is dead
}
solution = pd.DataFrame(submission)
solution.to_csv('submission.csv', index=False)